In [ ]:
import pandas as pd
from transformers import pipeline
from datasets import Dataset

# **加载 FinBERT（使用 GPU 加速）**
finbert = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone", device=0)

# **读取 Excel 数据**
df = pd.read_excel("updated_news_refinitiv.xlsx", engine="openpyxl")

# **去重 headline**
df_uni = df.drop_duplicates(subset=["headline"], keep="first")

# **转换为 Hugging Face Dataset（优化 GPU 计算）**
dataset = Dataset.from_pandas(df_uni)

# **批量推理（自动分批，提高 GPU 利用率）**
results = finbert(dataset["headline"], batch_size=32)

# **拆分结果**
df_uni["sentiment"] = [res["label"] for res in results]
df_uni["confidence"] = [res["score"] for res in results]

# **显示前 10 行**
print(df_uni.head(10))